In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import logging
import time
import lightgbm as lgb
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
import pickle

In [2]:
articles = pd.read_csv('../data/raw/articles.csv')
train_clicks = pd.read_csv('../data/raw/train_click_log.csv')
test_clicks = pd.read_csv('../data/raw/testA_click_log.csv')

In [3]:
articles.shape

(364047, 4)

In [4]:
train_clicks.shape

(1112623, 9)

In [5]:
test_clicks.shape

(518010, 9)

In [6]:
train_clicks['user_id'].unique().shape

(200000,)

In [7]:
test_clicks['user_id'].unique().shape

(50000,)

In [9]:
all_features = pd.read_csv('../temp/main_features.csv')

In [10]:
all_features.shape

(7500000, 31)